In [27]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [70]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-price aok-align-center'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()
        except:
            price ='None'

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available
#Function Description

def get_analysis(soup):

    try:
        # Outer Tag Object
        title = soup.find("ul", attrs={"class":'a-unordered-list a-vertical a-spacing-mini'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string
# def get_description(soup):
#     try:
#         # Tìm tất cả các phần tử li trong ul có class cụ thể
#         list_items = soup.find_all("li", attrs={"class": "a-spacing-mini"})
        
#         # Lấy văn bản từ từng li và thêm vào danh sách
#         description = "\n".join([item.text.strip() for item in list_items])

#     except AttributeError:
#         description = ""

#     return description


def get_description(soup):

    try:
        description = soup.find("div", attrs={'id':'productDescription'}).text.strip()

    except AttributeError:
        description = ""	

    return description

In [89]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=Laptop&s=exact-aware-popularity-rank&page=5&crid=XLIZ27GY4IBD&qid=1729247303&sprefix=laptop%2Caps%2C331&ref=sr_pg_5"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[], "about_it":[],"description":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))
        d['about_it'].append(get_analysis(new_soup))
        d['description'].append(get_description(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_25392\2907351117.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  amazon_df['title'].replace('', np.nan, inplace=True)


In [90]:
amazon_df

,title,price,rating,reviews,availability,about_it,description
0,"ASUS TUF Gaming A15 (2024) Gaming Laptop, 15.6...",$658.30,4.7 out of 5 stars,656 ratings,In Stock,READY FOR ANYTHING - Jump right into the actio...,The ASUS TUF Gaming A15 is a powerful gaming l...
1,Lexibook - Disney Stitch - Bilingual English/S...,$44.99,4.6 out of 5 stars,"2,875 ratings",In Stock,ENHANCE LEARNING POTENTIAL - Engage young mind...,Unleash Limitless Learning and Adventure with ...
2,JanSport Laptop Backpack - Computer Bag with 2...,$54.99,4.7 out of 5 stars,"19,836 ratings",In Stock,Go Big or Go Home: Always have room for your e...,These extra-large capacity backpacks for colle...
3,"HP 17 Laptop, 17.3” HD+ Display, 11th Gen Inte...",$381.17,4.3 out of 5 stars,192 ratings,Only 8 left in stock - order soon.,【High Speed RAM And Enormous Space】8GB high-ba...,Key Features and Benefits:CPU: 11th Gen Intel ...
4,"Lenovo 2023 V15 G3 15.6"" FHD Business Laptop C...",$475.75,4.3 out of 5 stars,48 ratings,Only 9 left in stock - order soon.,"CPU: 12th Gen Intel Core i5-1235U, 10C (2P + 8...",
5,"BERTASCHE Lunch Backpack, Laptop Backpack 15.6...",$41.49,4.3 out of 5 stars,226 ratings,Only 1 left in stock - order soon.,Detachable Insulated Lunch Box: This backpack ...,
6,ScratchPad Pro — Cardboard Laptop Cat Scratche...,$29.99,4.7 out of 5 stars,217 ratings,In Stock,"Cat Computer: Set up this creative cat toy, sh...",Cat got your...WFH setup? ScratchPad Pro is he...
7,"Targus 16 Inch Compact Rolling Backpack, Black...",$70.33,4.3 out of 5 stars,"6,934 ratings",In Stock,"Laptop Backpack for Work, Travel, or College: ...","The Targus 16"" Compact Rolling Backpack in sle..."
8,"LOVEVOOK Travel Backpack for Women, TSA Carry ...",$40.25,4.5 out of 5 stars,598 ratings,Only 5 left in stock - order soon.,Travel Backpack for Women: Crafted from premiu...,
9,"USB C Docking Station Dual Displayport, Selore...",$34.99,4.1 out of 5 stars,"3,077 ratings",In Stock,【7 IN 1 USB C Docking Station】Selore USB C to ...,
